In [ ]:
from scripts.scale_transform_magnitude import compute_stm, compute_stm_multi_channel

import mirdata
import librosa
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import numpy as np

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
groove_dataset = mirdata.initialize("groove_midi")
# groove_dataset.download()
groove_dataset.validate()

In [ ]:
f = []
l = []
for _, track, in tqdm(groove_dataset.load_tracks().items()):
    if track.audio_path != None and track.beat_type == "beat" and duration:
        try:
            y, sr = librosa.load(track.audio_path, sr=None) 
            stm = compute_stm(y, sr, with_padding=True, autocor_window_type="hamming")[:200]
            f.append(stm)
            l.append(track.style)
        except Exception as e:
            print(f"error: {e}")